In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

rcParams['figure.figsize'] = [10.0,8.0]


In [ ]:
fname = "f105w.fits"
hdu_list = fits.open(fname)
hdu_list.info()

In [ ]:
data = hdu_list[0].data

In [ ]:
print(type(data))
print(data.shape)

In [ ]:
image_header = hdu_list[0].header

In [ ]:
hdu_list.close()

In [ ]:
m = np.mean(data)
s = np.std(data)
plt.imshow(data, interpolation = 'nearest', cmap='gray',vmin=m-s,vmax=m+s,origin='lower')
plt.colorbar()

In [ ]:
data = data.byteswap().newbyteorder()
bkg = sep.Background(data)

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
bkg_image = bkg.back()

In [ ]:
plt.imshow(bkg_image, interpolation='nearest',cmap='gray',origin='lower')
plt.colorbar();

In [ ]:
bkg_rms = bkg.rms()

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest',cmap='gray',origin='lower')
plt.colorbar();

In [ ]:
data_sub = data - bkg

In [ ]:
objects = sep.extract(data_sub,1.5,err=bkg.globalrms)

In [ ]:
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation = 'nearest', cmap = 'gray', vmin=m-s, vmax=m+s,origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i],objects['y'][i]), width = 6*objects['a'][i],height=6*objects['b'][i],angle=objects['theta'][i]*180.0/np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

In [ ]:
print( "Sources found " , len(objects))

In [ ]:
objects.dtype.names

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'],objects['y'],3.0, err=bkg.globalrms,gain=1.0)

In [ ]:
for i in range(8640):
    print("object{:d}: flux = {:f}+/- {:f}".format(i, flux[i],fluxerr[i]))

In [ ]:
x = flux
plt.hist(x,range=[0,1],bins='auto')
plt.figure(figsize=(0.05,0.05))
plt.show()